In [1]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [34]:
from nba_api.stats.endpoints import teamyearbyyearstats

test = teamyearbyyearstats.TeamYearByYearStats(league_id="00", 
                                               per_mode_simple = "Totals", 
                                               season_type_all_star = "Regular Season", 
                                               team_id="1610612756")
test = test.get_data_frames()[0]
test = test[test.YEAR == '2020-21']
test.head()

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
52,1610612756,Phoenix,Suns,2020-21,72,51,21,0.708,2,0,...,630,2462,3092,1939,1374,517,902,312,8300,7


In [33]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'PHX'][0]
celtics_id = celtics['id']

from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2020':]

# Subset the games to when the last 4 digits of SEASON_ID were 2017.
games_2021 = games[games.SEASON_ID.str[-4:] == '2020']
games_2021.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42020,1610612756,PHX,Phoenix Suns,0042000312,2021-06-22,PHX vs. LAC,W,239,104,...,0.889,7,32,39,24,5,3,8,20,1.0
1,42020,1610612756,PHX,Phoenix Suns,0042000311,2021-06-20,PHX vs. LAC,W,240,120,...,1.000,7,36,43,31,5,3,7,16,6.0
2,42020,1610612756,PHX,Phoenix Suns,0042000234,2021-06-13,PHX @ DEN,W,241,125,...,0.935,7,41,48,18,6,6,14,21,7.0
3,42020,1610612756,PHX,Phoenix Suns,0042000233,2021-06-11,PHX @ DEN,W,239,116,...,0.900,4,34,38,26,10,2,12,13,14.0
4,42020,1610612756,PHX,Phoenix Suns,0042000232,2021-06-09,PHX vs. DEN,W,240,123,...,0.882,8,44,52,27,3,4,7,18,25.0


In [25]:
from nba_api.stats import 
get_endpoint_documentation(teamyearbyyearstats)

NameError: name 'get_endpoint_documentation' is not defined